<a href="https://colab.research.google.com/github/shelbygreen/leoncounty/blob/master/geomapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Single-use Plastic Regulations (SUPs) Map
Workflow:
0. Set-up Google Drive API and connect to Google Sheets
1. Extract relevant data from Google Sheet and put into a DataFrame
2. Further data manipulation/cleaning/processing



Resources:
1. Connect to Google Sheets
https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html


In [0]:
"""
INSTALL: install required libraries: gspread, oauth2client, mapboxgl
"""

"""
IMPORT: packages and file
"""
#import packages
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os
from mapboxgl.utils import *
from mapboxgl.viz import *
from getpass import getpass
import json
import geopandas as gpd

#use credentials to create a client to connect with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('./client_secret.json', scope)
client = gspread.authorize(creds)

#opens first sheet of the file 
sheet = client.open_by_key("1sVq7eYnQKXtDdW-5ioGtYrwrWGpXa-Bkyqv2HQaBPcU").get_worksheet(0)

#extract info and save as dataframe
attributes_df = pd.DataFrame(sheet.get_all_records())

#token for mapbox
token = getpass('Enter token here')

Enter token here··········


In [0]:
"""
MANIPULATION: cleaning, filtering, and mapping.
"""
#cleaning attributes dataframe
attributes_df["Locality"] = [x.split(' County')[0] for x in attributes_df["Locality"]]
attributes_df["Locality"][26] = "Dade"

#importing cities and counties geojson
#mapshaper.org and filter the cities geojson (filter '"...".indexOf(NAME) > -1')
cities_gdf = gpd.read_file('./cities.json')
counties_gdf = gpd.read_file('./counties.json').rename(columns = {'COUNTYNAME':'NAME'})

#merging cities and counties into one dataframe
localities_df = pd.concat([cities_gdf, counties_gdf], join="inner", ignore_index=True)
localities_df["NAME"] = localities_df["NAME"].str.title()

#merge localities and attributes on name
mapped_df = localities_df.merge(attributes_df, left_on="NAME", right_on="Locality")

#save df as geoJSON
mapped_df.to_file('./mappedregulations.geojson', driver='GeoJSON')

"""
for future reference: visualuzation/interactive map using mapbox
viz = ChoroplethViz('./mappedregulations.geojson', access_token=token)
viz.show()
"""